In [1]:
import re
import math
import sys

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.cluster import DBSCAN

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TrueCoffee.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 353
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 953 words
filter frequent words: 374 words
filter letter words: 372 words
filter stop words: 221 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### DBSCAN

In [6]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def predict_cluster(bow_corpus, eps, min_samples):
    model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(bow_corpus)
    predicted_labels = model.labels_
    predicted_labels += 1

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result

In [8]:
bow_corpus = get_bow(idx_corpus)
result = predict_cluster(bow_corpus, 0.42, 5)
bow_corpus.head()

,กาแฟ,ชง,พนง,ร้าน,สวย,เงิน,เรียน,ทรูมูฟ,ลูกค้า,สิทธิ์,...,มากกกก,สำหรับ,อินเตอร์เน็ต,จบ,สม,ห่วย,ไล่,แบล็คการ์ด,กาก,อ่า
0,0.125000,0.125,0.125,0.125000,0.25,0.125,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000,0.000,0.000000,0.00,0.000,0.000,0.2,0.2,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000,0.000,0.000000,0.00,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000,0.000,0.000000,0.00,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.181818,0.000,0.000,0.090909,0.00,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Result

In [9]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_clusters = len(label_count[0])
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([194,  12,  98,  10,   6,   8,   7,   5,   6,   7]))
		percent
0  |	194	1.0


In [10]:
comment_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1)

3 ช่วงนี้เนตช้ามากเลยครับ ช่วยปรับปรุงด้วยครับ รายเดือนแพงแต่ใช้ไม่คุ้ม
4 ไม่ ชอบ การ แบ่ง ชนชั้น ในร้านกาแฟ ผู้ถือบัตรอีกแบบ นั่งได้แค่ตรงนี้ ผู้ถือบัตรอีกแบบนั่งตรงนี้ จ่ายราคากาแฟก็เท่ากันไหม
6 ก่อนลูกค้าจะสั่ง รบกวนพนักงานอ้าปากบอกเลยค่ะ สิทธิ์เต็ม เบื่อมากกก
7 มี  มีโปรก็ไม่ได้ดืม สิทธิ์เต็ม ทำไงล่ะ ขอบคุณครับ ไปกินสตาบัคดีกว่า ไมอยากเป็นเหยื่อยการค้า
9 อร่อยดีเเล้วต้อง  นะครับอย่าลืมใช้ นะครับผม
10 ผมว่า ผมชอบรสชาติของกาแฟคุณ หลายๆแก้วประทับใจกว่า แบรนด์นางเงือกแต่ว่า แพงไปหว่ะ ไม่เหมาะกะคนไทย ถึงแม้จะใช้ส่วนลด ลูกค้าทรู แล้วก้ตามสังเกตได้ว่า น้อยสาขา ที่คนจะแน่น หรือต้องต่อคิวสัีงกาแฟ เหมือนแบรนด์นางเงือก
11 ใช้   ทีไรเต็มตลอด พนักงานแนะนำให้กด   ไว้ตอนเที่ยงคืน
12 ถ้า กาแฟจืด เหมือนน้ำเปล่า แสดงว่าชอบกิน สายพันธุ์โรบัสต้า เบอดี้3  1ถ้า  แพง ให้เปิด  มี 10 กว่าโปร จนไม่รู้จะใช้โปรไหน จน งง
13 ชาเย็นอร่อย ชอบที่เป็นน้ำแข็งปั่นละเอียด แต่ขอน้ำเยอะกว่านี้ได้ไหม ดูไปสองปื้ดหมดแหล่ว ราคาโหดร้ายไป๊
14 โปรติดสัญญานี่เนตใช้ดีมาก ดูยูทูปเท่าไหร่ก็ไม่หมด พอครบสัญญาปุ๊บ อีดอกแค่สไล้เฟชบุค 

In [11]:
token_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1)

3 ['เนต', 'ปรับปรุง', 'เดือน', 'แพง', 'คุ้ม']
4 ['ชอบ', 'ร้าน', 'กาแฟ', 'บัตร', 'นั่ง', 'บัตร', 'นั่ง', 'จ่าย', 'ราคา', 'กาแฟ', 'ไหม']
6 ['ลูกค้า', 'สั่ง', 'รบกวน', 'พนักงาน', 'สิทธิ์', 'เต็ม', 'กก']
7 ['โปร', 'ดืม', 'เต็ม', 'ทำ', 'กิน', 'สตาบัค', 'ดี', 'ไม']
9 ['อร่อย', 'ดี', 'อย่า', 'ผม']
10 ['ผม', 'ผม', 'ชอบ', 'รสชาติ', 'กาแฟ', 'แก้ว', 'แบรนด์', 'แพง', 'กะ', 'คน', 'ไทย', 'ลด', 'ลูกค้า', 'ทรู', 'สาขา', 'คน', 'คิว', 'เหมือน', 'แบรนด์', 'นางเงือก']
11 ['เต็ม', 'พนักงาน', 'กด', 'ตอน', 'คืน']
12 ['กาแฟ', 'จืด', 'เปล่า', 'ชอบ', 'กิน', 'แพง', 'โปร', 'รู้', 'โปร', 'งง']
13 ['ชาเย็น', 'อร่อย', 'ชอบ', 'น้ำ', 'แข็ง', 'ปั่น', 'น้ำ', 'ไหม', 'ดู', 'สอง', 'ราคา']
14 ['เนต', 'ดี', 'ดู', 'ไหร่', 'เดือน', 'หาย']
15 ['ทรู', 'บ้าน', 'คืน', 'จ่าย', 'พัน', 'เดือน', 'ปรับปรุง', 'ฝาก', 'ทรูคอฟฟี่']
17 ['ย้าย', 'ค่าย', 'สัญญาณ', 'วว', 'เกี่ยว', 'กาแฟ']
18 ['เหมือน', 'สาขา', 'รสชาติ', 'เหมือน', 'คน', 'ทำ', 'กาแฟ', 'รสชาติ', 'ลูกค้า', 'ทรู', 'สิทธิ์', 'ลด', 'พิเศษ']
19 ['เมนู', 'ขนม', 'ชอบ']
20 ['รสชาติ', 'เห